# Project: LSTM_module (predict)

학습된 모델로 음성파일 스팸 예측

## 순서

1. import: 필요한 모듈 import
2. 파일 불러오기: 모델 학습에 사용된 객체, 변수 등 불러오기
3. STT: 음성파일 STT변환
4. 전처리: 텍스트 토큰화
5. 예측: 스팸 탐지

## 모듈 import

In [178]:
import sys
import json
import pickle
import re
import numpy as np

import speech_recognition as sr

import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

## GPU 셋팅

GPU 사용 시 활성화

In [179]:
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

## 파일 이름 선언

필요 변수, 모델, 토큰 객체 등 파일 이름 선언

In [180]:
file_name = "LSTM_module_ver4.0"

## 파일 불러오기

tokenizer 객체 불러오기

In [181]:
with open(file_name+'_tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

변수 불러오기

In [182]:
with open(file_name+'_variable.json') as f:
    var = json.load(f)

불용어 단어 불러오기

In [183]:
with open(file_name+'_stopwords.json') as f:
    stopwords = json.load(f)

불러온 변수 초기화

In [184]:
min_data = var['min_data'] # 데이터 문자열 최소 길이
data_length = var['data_length'] # 데이터 문자열 최대 길이
split_list = var['split_list'] # 문장 단위로 분리하기 위한 글자 리스트
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)
trunc_type = var['trunc_type']
padding_type = var['padding_type']

print('min_data:',min_data)
print('data_length:',data_length)
print('max_len:',max_len)
print('trunc_type:',trunc_type)
print('padding_type:',padding_type)

min_data: 10
data_length: 100
max_len: 150
trunc_type: post
padding_type: post


## STT 변환

음성파일 STT변환
* `language`: 언어 설정
* 약 80~100MB 용량 제한

In [185]:
r = sr.Recognizer()
harvard = sr.AudioFile('spam2.flac') # 100MB 용량 제한
with harvard as source:
    audio = r.record(source)

try:
    t = r.recognize_google(audio, language='ko-KR')
except:
    t = '녹음 내용이 존재하지 않습니다.'
print(t)

## 텍스트 데이터 분리

**findIndex**: 문장이 끝나는 부분의 index값을 찾기위한 함수

In [187]:
def findIndex(data_list, split_list, start_index):
    index_list = []
    for i in split_list:
        index = data_list.find(i, start_index)
        index_list.append(index)
        
    index_list = [i for i in index_list if i not in [-1]]
    if index_list == []:
        return -1
    
    index = min(index_list)
    
    return index     

`data_length` 값 이후에 `split_list` 에 있는 문자가 나오면 자르기

In [188]:
t = re.sub("네|[^가-힣 ]", "", t) # 특수문자 제거
index = findIndex(t, split_list, data_length)

i = 0
t_list = []
while index != -1:
    x = t[i:index+2]
    t_list.append(x)

    i = index+2
    index = findIndex(t, split_list, i+data_length)
else:
    x = t[i:]
    if len(x) > min_data:
        t_list.append(x) # 텍스트 마지막 부분 추가

## 전처리: 토큰화 및 불용어 처리

Okt 객체 선언

In [189]:
okt = Okt()

**preTreatment**: 토큰화 및 불용어 처리 함수

In [190]:
def preTreatment(data):
    global stopwords
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp

**sentiment_predict**: 정수 인코딩 후 모델 예측 함수

In [191]:
def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(data, maxlen = max_len, truncating = trunc_type, padding = padding_type)
    score = float(model.predict(pad_new))
    print(score)
    print('----------------------')
    return score

## 모델 예측

학습된 모델 불러오기

In [192]:
model = tf.keras.models.load_model(file_name+'.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 150)         750300    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               100400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 850,801
Trainable params: 850,801
Non-trainable params: 0
_________________________________________________________________


분리한 각 데이터 확률 예측 후 평균 구하기

In [193]:
score_sum = 0.0
for txt in t_list:
    print(txt)
    data = preTreatment(txt)
    score_sum += sentiment_predict(data)
    
score_result = score_sum / len(t_list)

여보세요 들리냐 하나 둘 셋 넷 잘들려 음 뭔 얘기를 할까 이게 스팸으로 인식을 할라나 모르겠 한다면 하는거고 뭐 상관없지 스팸 인식 안됐으면 좋겠
0.0789751410484314
----------------------


## 예측 확률 출력

In [194]:
print("{:.2f}% 확률로 스팸입니다.\n".format(score_result * 100))

7.90% 확률로 스팸입니다.

